# 把这文件放在和mlearn/同一层文件夹运行
## 不然import会报错

In [2]:
import mlearn_cpu as mlearn
from mlearn_cpu import functional as F
from mlearn_cpu import layers
from mlearn_cpu.optimizers import SGD, RMSProp, Momentum
from torchvision import datasets,transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from mlearn_cpu import DataLoader
from mlearn_cpu import pre as P
import sys

# 加载数据集

In [3]:
train = datasets.MNIST("datasets", train=True, download=True,
                      transform = transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST("datasets", train=False, download=True,
                     transform = transforms.Compose([transforms.ToTensor()]))

pre = [P.normalize_MinMax]
trainset = DataLoader((train.data,train.targets),batch_size=32,shuffle=True,
                      preprocessing=pre)
testset = DataLoader((test.data, test.targets), batch_size=32, shuffle=True,
                    preprocessing=pre)

In [6]:
class Net(mlearn.Module):
    def __init__(self):
        self.dense1 = layers.Dense(784,300)
        self.dense2 = layers.Dense(300,150)
        self.dense3 = layers.Dense(150,64)
        self.dense4 = layers.Dense(64,10)
        
    def forward(self, inputs):
        o = self.dense1(inputs)
        o = F.relu(o)
        o = self.dense2(o)
        o = F.tanh(o)
        o = self.dense3(o)
        o = F.relu(o)
        o = self.dense4(o)
        o = F.tanh(o)
        return (F.relu(o))

In [10]:
hist = []
def train():
    net = Net()
    optimizer = SGD(net,0.001)
    EPOCHS = 1
    net.zero_grad()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        bar = " "*20
        for i, batch in enumerate(trainset, 0):
            features, labels = batch
            net.zero_grad()
            predict = net(features.reshape(-1,784))
            loss = F.cross_entropy(predict, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.data
            print(f"\r{epoch+1}/{EPOCHS} Batch %-4d/1874  [{bar}] -> Loss %.5f"%(i,loss.data), end="")
            sys.stdout.flush()
        print()
    print('trainning completed!')


In [11]:
%timeit train()

1/1 Batch 1874/1874  [                    ] -> Loss 1.80485
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.86110
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.82751
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.95530
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.75830
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.89484
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.86275
trainning completed!
1/1 Batch 1874/1874  [                    ] -> Loss 1.85977
trainning completed!
9.25 s ± 364 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
correct = 0
total = 0

for batch in testset:
    features, labels = batch
    o = net(features.reshape(-1,784))
    predict = []
    for x in o.data:
        predict.append(np.argmax(x))
    for b in predict == labels.data:
        if b:
            correct += 1
    total += 32
print("Accuracy %.5f"%(correct / total))

Accuracy 0.12600


In [ ]:
np.zeros_like((1,2))

In [ ]:

num_episodes = 60000
for i_episode in range(1, num_episodes + 1):
    # Print out which episode we're on, useful for debugging.
    if i_episode % 1000 == 0:
        print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
        sys.stdout.flush()


In [ ]:
import time
import sys

toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['

for i in range(toolbar_width):
    time.sleep(0.1) # do real work here
    # update the bar
    sys.stdout.write("-")
    sys.stdout.flush()

sys.stdout.write("]\n") # this ends the progress bar